In [1]:
import pandas as pd
import os

In [2]:
pd.options.display.max_rows=15

In [3]:
train_file_path = os.path.join(os.pardir, 'data', 'raw','train.csv')
test_file_path = os.path.join(os.pardir, 'data', 'raw', 'test.csv')

In [4]:
df_train = pd.read_csv(train_file_path)
df_test = pd.read_csv(test_file_path)

In [5]:
df_train

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
5,00004f9a462a357c33be,"Is Gaza slowly becoming Auschwitz, Dachau or T...",0
6,00005059a06ee19e11ad,Why does Quora automatically ban conservative ...,0
...,...,...,...
1306115,ffffc0c7158658a06fd9,How do I consume 150 gms protein daily both ve...,0
1306116,ffffc404da586ac5a08f,What are the good career options for a msc che...,0


In [6]:
df_train.target.value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [7]:
df_test

,qid,question_text
0,00014894849d00ba98a9,My voice range is A2-C5. My chest voice goes u...
1,000156468431f09b3cae,How much does a tutor earn in Bangalore?
2,000227734433360e1aae,What are the best made pocket knives under $20...
3,0005e06fbe3045bd2a92,Why would they add a hypothetical scenario tha...
4,00068a0f7f41f50fc399,What is the dresscode for Techmahindra freshers?
5,000a2d30e3ffd70c070d,How well are you adapting to the Trump era?
6,000b67672ec9622ff761,What should be the last thing people do in life?
...,...,...
56363,fffb418e5e087182294d,How can I make squid tender?
56364,fffc3df92967fc1ff426,Why did England become the biggest colonial po...


In [8]:
df_train.question_text.loc[df_train.target == 1]

22         Has the United States become the largest dicta...
30         Which babies are more sweeter to their parents...
110        If blacks support school choice and mandatory ...
114        I am gay boy and I love my cousin (boy). He is...
115                     Which races have the smallest penis?
119                        Why do females find penises ugly?
127        How do I marry an American woman for a Green C...
                                 ...                        
1306031    Why do Jews want endless immigration to the US...
1306071    Will Oprah buy the DNC to guarantee her nomina...
1306093    How is it to have intimate relation with your ...
1306094    Why is it when singers have lyrics about voice...
1306099                    Do pakis smell of curry and shit?
1306103    Isn't Trump right after all? Why should the US...
1306112                  Are you ashamed of being an Indian?
Name: question_text, Length: 80810, dtype: object

In [9]:
df_train.question_text.loc[0]

'How did Quebec nationalists see their province as a nation in the 1960s?'

In [10]:
df_train['length']=df_train.question_text.apply(lambda x : len(x.split()))

In [11]:
df_train['length']

0          13
1          16
2          10
3           9
4          15
5          10
6          18
           ..
1306115    15
1306116    14
1306117    17
1306118    19
1306119     4
1306120    13
1306121    11
Name: length, Length: 1306122, dtype: int64

In [17]:
df_train.groupby(['target'])['length'].mean()

target
0    12.508533
1    17.277812
Name: length, dtype: float64

In [19]:
df_train.groupby(['target'])['length'].median()

target
0    11
1    15
Name: length, dtype: int64

In [ ]:
df_train['question_text'].str.split().value_counts()